In [1]:
import pandas as pd
import numpy as np
import os 
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#TEST_IMAGE_DIR = "./dog-breed-identification/test/"
TRAIN_IMAGE_DIR = "./dog-breed-identification/train/"
LABELS = './dog-breed-identification/labels.csv'

data = pd.read_csv(LABELS)

2023-04-26 14:05:06.762961: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
#Data from the csv only contans the ids of the photos and not the photos itself
#Iterate through the data and check whether the photo is in test or train
#Then fetch it and store it in its proper variable
import tensorflow as tf
from os.path import join
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

def load_and_preprocess_images(image_dir, image_ids, target_size=(224, 224)):
    image_data = []
    for img_id in image_ids:
        img_path = join(image_dir, img_id + ".jpg")
        img = load_img(img_path, target_size=target_size)
        img_array = img_to_array(img)
        preprocessed_img = preprocess_input(img_array) #I dont flatten the image here
        image_data.append(preprocessed_img)
    return np.array(image_data)

# Get image IDs for train and test
train_image_ids = data['id'].values
labels = data['breed'].values
# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

train_image_ids, unfinished_test_image_ids, train_labels, unfinished_test_labels = train_test_split(train_image_ids, encoded_labels, test_size=0.3, random_state=42, stratify=encoded_labels)

test_image_ids, val_image_ids, test_labels, val_labels = train_test_split(unfinished_test_image_ids, unfinished_test_labels, test_size=0.5, random_state=42, stratify=unfinished_test_labels)

train_images = load_and_preprocess_images(TRAIN_IMAGE_DIR, train_image_ids)
test_images = load_and_preprocess_images(TRAIN_IMAGE_DIR, test_image_ids)
val_images = load_and_preprocess_images(TRAIN_IMAGE_DIR, val_image_ids)

In [3]:
train_images = train_images / 250.0
test_images = test_images / 250.0
val_images = val_images / 250.0

In [4]:
from tensorflow.keras.models import load_model

# Load the saved model
loaded_model = load_model('pre_trained_38.h5')

# Use the loaded model to make predictions
predictions = loaded_model.predict(test_images)

2023-04-26 14:05:48.099672: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


48/48 [==============================] - 226s 5s/step


In [5]:
test_loss, test_acc = loaded_model.evaluate(test_images,  test_labels, verbose=2)
print('\nTest accuracy:', test_acc)

48/48 - 223s - loss: 2.4605 - accuracy: 0.3810 - 223s/epoch - 5s/step

Test accuracy: 0.380952388048172


In [55]:
# shape of images
print(np.shape(test_images))

# reshape single image for prediction
test_img = np.reshape(test_images[110], (1, 224, 224, 3))

# After reshaping individual image can get label of single image
predict = loaded_model.predict(test_img)
predicted_class_index = np.argmax(predict)
predicted_label = label_encoder.inverse_transform([predicted_class_index])

print('Predicted breed:', predicted_label[0])

(1533, 224, 224, 3)
1/1 [==============================] - 0s 210ms/step
Predicted breed: leonberg


In [45]:
from sklearn.metrics import precision_score, recall_score, f1_score

# print('precision_score from out model: %s' % (precision_score(test_labels, predictions)))
# print('recall_score from out model: %s' % (recall_score(predictions, test_labels)))
# print('f1_score from out model: %s' % (f1_score(predictions, test_labels)))

ValueError: Classification metrics can't handle a mix of multiclass and continuous-multioutput targets